# Optimización de Procesos de Inversión
<img style="float: right; margin: 15px 15px 15px 15px;" src="https://upload.wikimedia.org/wikipedia/commons/d/db/Logo_ITESO_normal.jpg" title="git" width="180" height="50">

**Proyecto de Aplicación Profesional**

#### Mtro. Nicolás Vázquez González


### Equipo 1
##### CASTILLO MEDA, LUCÍA
##### GUTIÉRREZ VALDIVIA, JUAN CARLOS
##### OROZCO GÓMEZ, DANIEL
##### PONCE ANGUIANO, CARLOS DANIEL
##### SÁNCHEZ PASZKO, JOSÉ WALDEN


## Primera Entrega
**Definición y Alcance del Proyecto**


### Proyecto 1: Selección y Evaluación de Activos

> Objetivo: Automatizar el análisis bursátil en la selección de activos financieros.


<div class="alert alert-block alert-info">
<b>Introducción</b>
</div>

La constante evolución de los mercados financieros y la creciente complejidad de la toma de decisiones de inversión han generado la necesidad de adoptar enfoques innovadores que optimicen la precisión y la eficiencia en la gestión de activos financieros. En este contexto, el presente proyecto surge como una propuesta ambiciosa para abordar esta problemática, a través de la implementación programática de un modelo matemático que integra el análisis fundamental con las tecnologías de vanguardia.

La aplicación resultante de este proyecto se presenta como una solución innovadora que contribuirá a brindar a los profesionales del ámbito financiero una herramienta sólida y confiable para la toma de decisiones de inversión. Al aprovechar la potencia de la tecnología y los avances en el campo del análisis de datos, se espera que los usuarios puedan contar con información objetiva y precisa, respaldada por el rigor del análisis fundamental y la robustez de los modelos matemáticos.

Es así que este proyecto de aplicación profesional propone una solución orientada a optimizar las decisiones de inversión en activos financieros a través de la implementación de un modelo matemático que combina el análisis fundamental con las capacidades tecnológicas actuales. La contribución de esta iniciativa reside en su potencial para revolucionar la manera en que los profesionales abordan la gestión de activos financieros, permitiéndoles tomar decisiones más informadas, eficientes y libres de sesgos cognitivos.

<div class="alert alert-block alert-info">
<b>Definición del Proyecto</b>
</div>

El proyecto tiene como objetivo principal la creación de una aplicación profesional altamente especializada para la toma de decisiones de inversión en activos financieros. Para lograr esto, se empleará un enfoque de análisis fundamental, que se basa en la evaluación exhaustiva de diversos factores clave. Esto incluye el análisis de los estados financieros de las empresas, lo que permitirá una comprensión profunda de su salud económica y solidez financiera.

Se utilizarán modelos y algoritmos que permitirán procesar grandes volúmenes de datos de manera eficiente y realizar análisis complejos en tiempo real. Uno de los objetivos clave es reducir significativamente el tiempo que los profesionales de inversión requieren para tomar decisiones informadas. Además, al utilizar un enfoque basado en datos y análisis cuantitativos, se busca eliminar el sesgo cognitivo inherente a las decisiones de inversión, lo que podría mejorar significativamente la calidad de las decisiones tomadas.

Este proyecto busca revolucionar la forma en que se toman decisiones de inversión al combinar la experiencia humana con la potencia de la tecnología y la inteligencia artificial. En caso de encontrar un modelo que efectivamente sea capaz de emitir recomendaciones de inversión con base en los fundamentales, la aplicación resultante se convertirá en una herramienta esencial para los profesionales financieros, permitiéndoles tomar decisiones más fundamentadas y precisas en un mercado cada vez más complejo y dinámico.

<div class="alert alert-block alert-info">
<b>Conceptos Básicos </b>
</div>

Con el propósito de establecer una base sólida para comprender las generalidades y alcances del proyecto, se presenta a continuación una sección dedicada a la definición de conceptos fundamentales en el ámbito financiero y tecnológico. Estos conceptos servirán como cimientos para una comprensión más profunda de la implementación programática del modelo matemático y el análisis fundamental en la toma de decisiones de inversión. Es entonces que a continuación se definen los siguientes tecnicismos:


- *Análisis Fundamental:* Representa un enfoque exhaustivo que busca calcular el valor real de un título financiero a través de la evaluación detallada de los balances y estados financieros de una entidad. Este método compara este valor calculado con el precio actual de mercado del activo, con el fin de identificar posibles situaciones de infravaloración en un momento dado. Es importante tener en cuenta que tanto los resultados financieros como los precios experimentan cambios constantes, lo que implica que la cotización de un activo puede ser influenciada por una diversidad de factores, incluyendo los de naturaleza política y económica. El análisis fundamental se erige como una herramienta de gran relevancia para evaluar el riesgo financiero asociado a empresas, haciendo uso de un análisis minucioso del entorno en el que operan, la determinación de ratios financieros clave y la valoración intrínseca de las propias compañías. Es importante destacar que este enfoque se diferencia del análisis técnico, que se centra principalmente en el estudio de tendencias de mercado y gráficos bursátiles.


- *Activo:* En el ámbito bursátil, se refiere a un instrumento que representa un valor económico y puede ser objeto de negociación en los mercados financieros. Estos activos representan derechos de propiedad o de deuda sobre una entidad, como una empresa, gobierno u otra entidad emisora. Los activos financieros pueden variar en naturaleza y características, incluyendo acciones, bonos, opciones, futuros, divisas y otros instrumentos financieros.


- *Acción:* Representa una parte de la propiedad en la empresa emisora y otorga al titular el derecho a participar en las decisiones de la empresa y recibir una parte proporcional de las ganancias en forma de dividendos.


- *Indices bursátiles:* medidas estadísticas que reflejan el rendimiento general de un grupo de activos financieros, como acciones o bonos, en un mercado financiero específico. Calculados mediante fórmulas ponderadas, estos brindan una visión general del comportamiento del mercado y ayudan a los inversores a comprender cómo se está desempeñando un grupo de acciones en relación con un período anterior o un mercado más amplio. Ejemplos incluyen el S&P 500 y el Dow Jones en EE. UU.


- *Liquidez:* Se refiere a la facilidad con la que un activo financiero puede ser comprado o vendido en el mercado sin afectar significativamente su precio. Los activos líquidos son aquellos que pueden ser convertidos rápidamente en efectivo sin una disminución sustancial en su valor.


- *Riesgo de mercado:* Se define como la posibilidad de pérdidas en el valor de inversiones debido a cambios generales en los mercados financieros, como fluctuaciones en precios de activos, tasas de interés o tipos de cambio. Es un riesgo no diversificable y se vincula a factores macroeconómicos que afectan ampliamente a las inversiones, independientemente de su naturaleza. Este riesgo es una preocupación inherente para los inversores, ya que los movimientos en los precios de los activos financieros pueden impactar negativamente el valor de una inversión, incluso si los activos en cuestión tienen fundamentos sólidos. El riesgo de mercado puede influir en activos de diferentes clases y sectores, lo que subraya la importancia de gestionarlo de manera efectiva. Los inversores abordan este riesgo mediante estrategias de asignación de activos, diversificación y, en algunos casos, mediante el uso de instrumentos financieros de cobertura, como futuros o opciones


- *Sentimiento de mercado:* Actitud general o percepción emocional de los inversores y participantes de mercados financieros. Puede ser positivo, negativo o neutral, y a menudo influye en las decisiones de compra y venta de los inversores. Se basa en factores como noticias, tendencias económicas y opiniones públicas, y puede tener un impacto en la volatilidad de los precios y en las tendencias del mercado. 


- *Ciclo económico:* Es la fluctuación periódica de la actividad económica en una economía, que incluye fases de expansión y contracción. Las fases incluyen el auge, la recesión, la depresión y la recuperación, y están influenciadas por factores económicos y políticos.


- *Parsimonía:* En el contexto del análisis y modelado, alude al principio de que un modelo simple que explica los datos de manera efectiva es preferible a un modelo más complejo si no proporciona una mejora sustancial en la comprensión o la predicción.


- *Aprendizaje Supervisado:* Es un enfoque dentro del campo del machine learning donde se entrena un modelo utilizando un conjunto de datos etiquetados, es decir, datos donde la respuesta o el resultado deseado está previamente conocido. El modelo utiliza esta información para aprender a hacer predicciones o tomar decisiones sobre nuevos datos no vistos previamente. En el contexto financiero, esto podría implicar la creación de un modelo que "aprende" a hacer recomendaciones de inversión al comparar los datos históricos de activos financieros con sus resultados reales de rendimiento.

<div class="alert alert-block alert-info">
<b>Flujo de Trabajo del Proyecto</b>
</div>

Para lograr la implementación exitosa de este proyecto, se establecerá un flujo de trabajo riguroso y estructurado que abarcará desde la recopilación y procesamiento de datos hasta la generación y evaluación de recomendaciones de inversión. El proceso comenzará con la obtención de datos financieros relevantes, que posteriormente serán sometidos a un proceso de limpieza y transformación para asegurar su calidad y consistencia. A continuación, se desarrollarán modelos matemáticos avanzados que analizarán de manera integral los factores clave que influyen en la valoración de los activos financieros. Estos modelos permitirán generar recomendaciones basadas en evidencia objetiva y establecerán criterios cuantitativos para la toma de decisiones. Finalmente, se implementará una interfaz de usuario intuitiva que presentará de manera clara y concisa las recomendaciones generadas. Este enfoque de flujo de trabajo garantiza la coherencia y la precisión en todas las etapas del proceso, brindando confiabilidad y eficacia a la aplicación desarrollada.

A continuación, se muestra un esquema que visualiza de manera clara el flujo de trabajo propuesto para la implementación del proyecto; más delante se encuentra también la asignación de las tareas a cada una de las entregas que se tienen estipuladas a lo largo del proyecto:
<img style="center" src="https://ibm.box.com/shared/static/oeyhn9eiz48mf8m5hpf53kzo8upv24ba.jpeg" width="500px" height="200px" alt="atom" />

- **1er avance:** Algoritmo que descargue fundamentales de las compañías (pasos 1-3 del diagrama).
    - Definir activos y fundamentales a utilizar
    - Diseñar algoritmo para descargar los fundamentales 
    - Acomodar los datos recabados con miras a la fase de modelación

    
- **2do avance:** Implementación de análisis fundamental y demás tipos de análisis (pasos 4-9 del diagrama).
    - Evaluar rendimiento de las acciones seleccionadas.
    - Asignar "1" o "0" dependiendo del rendimiento.
    - Separar los datos en entrenamiento y prueba.
    - Realizar distintos modelos de clasificación y evaluarlos.
    - Seleccionar del mejor modelo.
 
 
- **3er avance:** Análisis y backtesting (pasos 10-12 del diagrama).
    - Realizar el análisis de significancia de las variables involucradas en el modelo.
    - Simular varios escenarios de backtesting para comprobar la eficiencia del modelo.
    
Los avances planteados con anterioridad corresponden a las entregas 2, 3 y 4, respectivamente, pues existen elementos adicionales a considerar para el transcurso de las prácticas profesionales. Tomando esto en consideración, a continuación se muestra un calendario propuesto para realizar la totalidad de las actividades en el tiempo que se tiene estipulado:
<img style="center" src=https://ibm.box.com/shared/static/kymna81s6p490gdogtski2r2qvw7vo8u.png alt="atom" />

<div class="alert alert-block alert-info">
<b>Documentacion, Herramientas y Recursos </b>
</div>

- **Descarga de Fundamentales: [Alpha Vantage API](https://www.alphavantage.co/documentation/)**
     - Free stock API service covering the majority of our datasets for up to 5 API requests per minute and 100 requests per day.
 - **Descarga de Precios: [Yahoo! Finance's API](https://pypi.org/project/yfinance/)**
 - **Trabajo Colaborativo: [GitHub](https://github.com)**
 - **Python**

## Entrega 2

In [12]:
# Importar librerías 
import yfinance as yf
import pandas as pd
import numpy as np
import Functions
import requests
import time
import os

In [61]:
# Cargar Datos
income_statement = Functions.load_full_excel('Data/Income/Income_Statement.xlsx')
balance_statement = Functions.load_full_excel('Data/Balance/Balance_Statement.xlsx')
sp500 = list(np.unique(np.array([income_statement.columns[i][0] for i in range(len(income_statement.columns))])))
prices = yf.download(tickers=sp500, start='2018-09-01')['Adj Close']

[*********************100%***********************]  388 of 388 completed


In [118]:
fiscal_endings = list(balance_statement[balance_statement.columns[0]].values)
balance_cols = ['fiscalDateEnding','currentDebt','inventory','totalAssets','totalCurrentAssets','currentAccountsPayable','currentNetReceivables','commonStockSharesOutstanding']
income_cols = ['totalRevenue','costofGoodsAndServicesSold','costOfRevenue']

In [98]:
balance_statement['AMCR'][balance_cols].set_index('fiscalDateEnding').loc[fiscal_endings[:-2]]

Fundamental,currentDebt,inventory,totalAssets,totalCurrentAssets,currentAccountsPayable,currentNetReceivables,commonStockSharesOutstanding
fiscalDateEnding,,,,,,,
2023-06-30,93000000,2083000000,17003000000,5308000000,2690000000,1875000000,1493000000
2023-03-31,209000000,2420000000,17278000000,5588000000,2528000000,2034000000,1470000000
2022-12-31,62000000,2509000000,17475000000,5863000000,2785000000,1972000000,1475000000
2022-09-30,76000000,2590000000,17292000000,5849000000,2839000000,1984000000,1474000000
2022-06-30,150000000,2439000000,17426000000,5853000000,3073000000,1935000000,1505000000
2022-03-31,72000000,2420000000,17987000000,6170000000,2816000000,2090000000,1503000000
2021-12-31,121000000,2273000000,17138000000,5391000000,2743000000,1889000000,1520000000
2021-09-30,68000000,2113000000,17044000000,5279000000,2412000000,1938000000,1530000000
2021-06-30,103000000,1991000000,17188000000,5266000000,2574000000,1864000000,1536800000


In [123]:
# Formatos de Fecha homogeneos
balance = balance_statement[sp500[0]][balance_cols].set_index('fiscalDateEnding')
income = income_statement[sp500[0]][income_cols]
income['fiscalDateEnding'] = balance.index.values
income = income.set_index('fiscalDateEnding')
# Unir columnas
company = pd.concat([balance,income], axis = 1)
company.columns = pd.MultiIndex.from_tuples( [(sp500[0],value) for value in company.columns.values] )

In [174]:
def quarters(dates_data):
    data = pd.Series(dates_data).dropna().values
    dates = [datetime.strptime(value,'%Y-%m-%d') for value in data]
    return np.array([date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.QuarterEnd() for date in  dates])

In [175]:
# Formatos de Fecha homogeneos
balance = balance_statement[sp500[0]][balance_cols]
balance['fiscalDateEnding'] = quarters(balance['fiscalDateEnding'].values)
balance = balance.set_index('fiscalDateEnding')
income = income_statement[sp500[0]][income_cols]
income['fiscalDateEnding'] = balance.index.values
income = income.set_index('fiscalDateEnding')
# Unir columnas
company = pd.concat([balance,income], axis = 1)
company.columns = pd.MultiIndex.from_tuples( [(sp500[0],value) for value in company.columns.values] )
for ticker in sp500[1:]:
    print(ticker)
    # Formatos de Fecha homogeneos
    balance = balance_statement[ticker][balance_cols]
    balance['fiscalDateEnding'] = quarters(balance['fiscalDateEnding'].values)
    balance = balance.set_index('fiscalDateEnding')
    income = income_statement[ticker][income_cols]
    income['fiscalDateEnding'] = balance.index.values
    income = income.set_index('fiscalDateEnding')
    # Unir columnas
    company = pd.concat([balance,income], axis = 1)
    company.columns = pd.MultiIndex.from_tuples( [(ticker,value) for value in company.columns.values] )
    companies = pd.concat([companies,company], axis=1)
companies

AAL
AAP


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [34]:
# '2022-12-31', '2019-06-30', '2019-03-31', '2018-09-30'
#prices.loc['2022-12-30']

Recomienda Nico:

PER

$$ PER = \frac{Precio De Accion}{\frac{Ingresos Netos}{Número De Acciones}} $$

Price Book Value

$$ PBV = \frac{Número de Acciones * Precios}{Activo Total} $$

Prueba del acido (Liquidez a Corto Plazo)

$$ Acido = \frac{Activo Circulante - Inventarios}{Pasivo Circulante} $$

Rotacion de los activos

$$ RA = \frac{Ventas Netas}{Activo Total} $$

Ciclo de conversión del efectivo

$$ CCE = \frac{365}{\frac{Costo Venta}{Inventario}} + \frac{365}{\frac{Ventas}{CXC}} - \frac{365}{\frac{Costo Venta}{CXP}} $$